In [2]:
from belly import *


In [57]:
spark.read.parquet('gs://spain-tweets/datalake').where('year=2020 and week=3').count()

3387

In [58]:
spark.read.parquet('gs://spain-tweets/datalake').where('year=2020 and week=4').count()

16613

In [21]:
spark.read.parquet('gs://spain-tweets/datalake').where('year=2020 and week=3').count()

3387

In [22]:
spark.read.parquet('gs://spain-tweets/datalake').where('year=2020 and week=4').count()

16613

In [513]:
import pickle
from gcsfs import GCSFileSystem

new_schema = create_schema()
fs = GCSFileSystem(project='trollhunters')

with fs.open('gs://spain-tweets/schemas/tweet-clean.pickle', 'wb') as f:
    pickle.dump(new_schema, f)